In [2]:
import torch
import argparse
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

import Ipynb_importer
from ReadingData import DataReader, Metapath2vecDataset
from model import SkipGramModel

importing Jupyter notebook from ReadingData.ipynb
importing Jupyter notebook from model.ipynb


In [5]:
class Metapath2VecTrainer:
    def __init__(self, file, min_count, window_size, batch_size, output_file, dim, iterations, initial_lr):
        self.data = DataReader(file, min_count)
        dataset = Metapath2vecDataset(self.data, window_size)
        self.dataloader = DataLoader(dataset, batch_size=batch_size,
                                     shuffle=True, num_workers=4, collate_fn=dataset.collate)

        self.output_file_name = output_file
        self.emb_size = len(self.data.word2id)
        self.emb_dimension = dim
        self.batch_size = batch_size
        self.iterations = iterations
        self.initial_lr = initial_lr #learning rate
        self.skip_gram_model = SkipGramModel(self.emb_size, self.emb_dimension)

        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")
        if self.use_cuda:
            self.skip_gram_model.cuda()

    def train(self):

        for iteration in range(self.iterations):
            print("\n\n\nIteration: " + str(iteration + 1))
            optimizer = optim.SparseAdam(self.skip_gram_model.parameters(), lr=self.initial_lr)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(self.dataloader))

            running_loss = 0.0
            for i, sample_batched in enumerate(tqdm(self.dataloader)):

                if len(sample_batched[0]) > 1:
                    pos_u = sample_batched[0].to(self.device)
                    pos_v = sample_batched[1].to(self.device)
                    neg_v = sample_batched[2].to(self.device)

                    scheduler.step()
                    optimizer.zero_grad()
                    loss = self.skip_gram_model.forward(pos_u, pos_v, neg_v)
                    loss.backward()
                    optimizer.step()

                    running_loss = running_loss * 0.9 + loss.item() * 0.1
                    if i > 0 and i % 500 == 0:
                        print(" Loss: " + str(running_loss))

            self.skip_gram_model.save_embedding(self.data.id2word, self.output_file_name)


In [8]:
file = '../output/output_path.txt'
min_count = 5   #单词频率截断
window_size = 5  #窗口大小
batch_size = 128  #批大小
output_file = '../output/embeddings.txt' #embedding输出路径
dim = 128  # embedding维度
iterations = 10  # 循环次数
initial_lr = 0.01  #learning rate

In [9]:
m2v = Metapath2VecTrainer(file, min_count, window_size, batch_size, output_file, dim, iterations, initial_lr)
m2v.train()

Total embeddings: 20864


  0%|          | 0/546 [00:00<?, ?it/s]




Iteration: 1


/home/linbang/anaconda3/envs/DGL/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
 92%|█████████▏| 502/546 [01:13<00:06,  6.83it/s]

 Loss: 2.9129858167833333


...

  0%|          | 0/546 [00:00<?, ?it/s]




Iteration: 10


 92%|█████████▏| 502/546 [00:59<00:04, 10.60it/s]

 Loss: 3.4719487614076523


100%|██████████| 546/546 [01:03<00:00,  8.60it/s]
